In [1]:
library(wooldridge)
library(stargazer)
library(knitr)


Please cite as: 

 Hlavac, Marek (2018). stargazer: Well-Formatted Regression and Summary Statistics Tables.
 R package version 5.2.2. https://CRAN.R-project.org/package=stargazer 



In [5]:
install.packages("prais")

Installing package into 'C:/Users/oldyu/R/win-library/3.5'
(as 'lib' is unspecified)


package 'prais' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\oldyu\AppData\Local\Temp\RtmpMxXRQ0\downloaded_packages


# Example 12.4: Prais-Winsten Estimation in the Event Study

In [3]:
data("barium")

barium_model <- lm(lchnimp ~ lchempi + lgas + lrtwex + befile6 + affile6 + afdec6, 
                   data = barium)


In [7]:
stargazer(type = "text", barium_model, single.row = TRUE, header = FALSE, digits=5)



                        Dependent variable:    
                    ---------------------------
                              lchnimp          
-----------------------------------------------
lchempi                3.11719*** (0.47920)    
lgas                     0.19635 (0.90662)     
lrtwex                  0.98302** (0.40015)    
befile6                  0.05957 (0.26097)     
affile6                 -0.03241 (0.26430)     
afdec6                  -0.56524* (0.28584)    
Constant               -17.80300 (21.04537)    
-----------------------------------------------
Observations                    131            
R2                            0.30486          
Adjusted R2                   0.27123          
Residual Std. Error     0.59735 (df = 124)     
F Statistic          9.06365*** (df = 6; 124)  
Note:               *p<0.1; **p<0.05; ***p<0.01


In [8]:
# Load the prais package, use the prais_winsten function to estimate.

library(prais)
barium_prais_winsten <- prais_winsten(lchnimp ~ lchempi + lgas + lrtwex + befile6 + affile6 + afdec6, data = barium)


Iteration 0: rho = 0
Iteration 1: rho = 0.2708
Iteration 2: rho = 0.291
Iteration 3: rho = 0.293
Iteration 4: rho = 0.2932
Iteration 5: rho = 0.2932
Iteration 6: rho = 0.2932
Iteration 7: rho = 0.2932


In [9]:
barium_prais_winsten


Call:
prais_winsten(formula = lchnimp ~ lchempi + lgas + lrtwex + befile6 + 
    affile6 + afdec6, data = barium)

Coefficients:
(Intercept)      lchempi         lgas       lrtwex      befile6      affile6  
  -37.07770      2.94095      1.04638      1.13279     -0.01648     -0.03316  
     afdec6  
   -0.57681  

AR(1) Coefficient rho: 0.2932


# Example 12.8: Heteroskedasticity and the Efficient Markets Hypothesis
These are Wednesday closing prices of value-weighted NYSE average, available in many publications. Wooldridge does not recall the particular source used when he collected these data at MIT, but notes probably the easiest way to get similar data is to go to the NYSE web site, www.nyse.com.

$$return_t = \beta_0 + \beta_1return_{t-1} + \mu_t$$

In [10]:
data("nyse")
?nyse 
return_AR1 <-lm(return ~ return_1, data = nyse)

$$\hat{\mu^2_t} = \beta_0 + \beta_1return_{t-1} + residual_t$$

In [11]:
return_mu <- residuals(return_AR1)
mu2_hat_model <- lm(return_mu^2 ~ return_1, data = return_AR1$model)
stargazer(type = "text",return_AR1, mu2_hat_model,  single.row = TRUE, header = FALSE, digits=5)


                                          Dependent variable:           
                               -----------------------------------------
                                     return             return_mu2      
                                       (1)                  (2)         
------------------------------------------------------------------------
return_1                        0.05890 (0.03802)  -1.10413*** (0.20140)
Constant                       0.17963** (0.08074) 4.65650*** (0.42768) 
------------------------------------------------------------------------
Observations                           689                  689         
R2                                   0.00348              0.04191       
Adjusted R2                          0.00203              0.04052       
Residual Std. Error (df = 687)       2.11040             11.17847       
F Statistic (df = 1; 687)            2.39946            30.05460***     
Note:                                        *p<0.

# Example 12.9: ARCH in Stock Returns
$$\hat{\mu^2_t} = \beta_0 + \hat{\mu^2_{t-1}} + residual_t$$

We still have return_mu in the working environment so we can use it to create $\hat{\mu^2_t}$, (mu2_hat) and $\hat{\mu^2_{t-1}}$ (mu2_hat_1). Notice the use R's matrix subset operations to perform the lag operation. We drop the first observation of mu2_hat and squared the results. Next, we remove the last observation of mu2_hat_1 using the subtraction operator combined with a call to the NROW function on return_mu. Now, both contain $688$ observations and we can estimate a standard linear model.

In [12]:
mu2_hat  <- return_mu[-1]^2
mu2_hat_1 <- return_mu[-NROW(return_mu)]^2
arch_model <- lm(mu2_hat ~ mu2_hat_1)
stargazer(type = "text",arch_model, single.row = TRUE, header = FALSE, digits=5)


                        Dependent variable:    
                    ---------------------------
                              mu2_hat          
-----------------------------------------------
mu2_hat_1              0.33706*** (0.03595)    
Constant               2.94743*** (0.44023)    
-----------------------------------------------
Observations                    688            
R2                            0.11361          
Adjusted R2                   0.11231          
Residual Std. Error     10.75907 (df = 686)    
F Statistic          87.92263*** (df = 1; 686) 
Note:               *p<0.1; **p<0.05; ***p<0.01
